In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir data
!cd data && wget "http://164.68.116.174:5000/download/files/isep/a3/deep-learning/respiratory-sound/extracted_features.npy"
!cd data && wget "http://164.68.116.174:5000/download/files/isep/a3/deep-learning/respiratory-sound/labels.npy"

mkdir: cannot create directory ‘data’: File exists
--2020-12-21 22:30:52--  http://164.68.116.174:5000/download/files/isep/a3/deep-learning/respiratory-sound/extracted_features.npy
Connecting to 164.68.116.174:5000... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126886528 (121M) [application/octet-stream]
Saving to: ‘extracted_features.npy’

extracted_features. 100%[===================>] 121.01M  22.7MB/s    in 5.9s    

2020-12-21 22:30:58 (20.4 MB/s) - ‘extracted_features.npy’ saved [126886528/126886528]

--2020-12-21 22:30:58--  http://164.68.116.174:5000/download/files/isep/a3/deep-learning/respiratory-sound/labels.npy
Connecting to 164.68.116.174:5000... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51648 (50K) [application/octet-stream]
Saving to: ‘labels.npy’

labels.npy          100%[===================>]  50.44K   264KB/s    in 0.2s    

2020-12-21 22:30:59 (264 KB/s) - ‘labels.npy’ saved [51648/51648]



In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
features, labels = np.load('/content/data/extracted_features.npy'), np.load('/content/data/labels.npy')
features = np.delete(features, np.where((labels == 'Asthma') | (labels == 'LRTI'))[0], axis=0)
labels = np.delete(labels, np.where((labels == 'Asthma') | (labels == 'LRTI'))[0], axis=0)

In [ ]:
le = LabelEncoder()
i_labels = le.fit_transform(labels)
oh_labels = to_categorical(i_labels)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, oh_labels, stratify=oh_labels, test_size=0.2, random_state=42)
num_rows, num_columns, num_channels = 40, 862, 1  # input shape

In [ ]:
num_labels = oh_labels.shape[1]
filter_size = 2

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=filter_size, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 39, 861, 16)       80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 430, 16)       0         
_________________________________________________________________
dropout (Dropout)            (None, 19, 430, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 18, 429, 32)       2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 214, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 214, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 213, 64)        8

In [ ]:
num_epochs = 200
num_batch_size = 128

callbacks = [
    ModelCheckpoint(
        filepath='mymodel2_{epoch:02d}.h5',
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1)
]

In [ ]:
model.fit(np.expand_dims(x_train, axis=3), y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(np.expand_dims(x_test, axis=3), y_test), callbacks=callbacks, verbose=1)

Epoch 1/200
6/6 [==============================] - ETA: 0s - loss: 6.8051 - accuracy: 0.6426
Epoch 00001: val_accuracy improved from -inf to 0.86413, saving model to mymodel2_01.h5
6/6 [==============================] - 1s 195ms/step - loss: 6.8051 - accuracy: 0.6426 - val_loss: 1.8576 - val_accuracy: 0.8641
Epoch 2/200
5/6 [========================>.....] - ETA: 0s - loss: 3.0248 - accuracy: 0.7797
Epoch 00002: val_accuracy did not improve from 0.86413
6/6 [==============================] - 0s 63ms/step - loss: 2.9993 - accuracy: 0.7899 - val_loss: 1.3369 - val_accuracy: 0.8641
Epoch 3/200
6/6 [==============================] - ETA: 0s - loss: 1.9929 - accuracy: 0.7940
Epoch 00003: val_accuracy did not improve from 0.86413
6/6 [==============================] - 0s 63ms/step - loss: 1.9929 - accuracy: 0.7940 - val_loss: 1.2844 - val_accuracy: 0.8587
Epoch 4/200
5/6 [========================>.....] - ETA: 0s - loss: 1.6634 - accuracy: 0.8594
Epoch 00004: val_accuracy did not improve fro

In [ ]:
model.save('/content/drive/My Drive/ISEP/A3/DEEP LEARNING/Respiration Sound/model.h5', save_format='h5')

In [ ]:
model = load_model('/content/drive/My Drive/ISEP/A3/DEEP LEARNING/Respiration Sound/model.h5')